# Connecting to SQL Server

## Introduction

The goal of this notebook is to attempt to connect to a Microsoft SQL Server instance, using AdventureWorks' database, to pull information from it.

## Dependencies

If running this on Linux/OSX, then unixODBC and the microsoft sql drivers must be installed  Your installation files should be something along the following:

### /etc/odbcinst.ini

```
[ODBC Driver 17 for SQL Server]
Description=Microsoft ODBC Driver 17 for SQL Server
Driver=/opt/microsoft/msodbcsql/lib64/libmsodbcsql-17.4.so.2.1
UsageCount=1
```

### /etc/odbc.ini

```
[SQLTDT]
Driver	= ODBC Driver 17 for SQL Server
Description    = Connection to local MSSQL Server (2017)
Trace	       = No
Server	       = SQL.TDT
```

Along with the above, for security considerations, the credentials are stored at `~/.creds/sql.txt` (your home directory).  Do not include passwords in these cells because of leaking credentials to the outside world.

In [1]:
from sqlalchemy import create_engine
from os.path import expanduser
import pyodbc
import pandas as pd

In [2]:
sqlcreds = open(F"{expanduser('~')}/.creds/sql.txt", 'r').read().strip()
connection_string = F"mssql+pyodbc://{sqlcreds}@SQLTDT"
# print(F"Connection String: {connection_string}")             

In [3]:
engine = create_engine(connection_string) 
connection = engine.connect()
connection.execute("use AdventureWorks")

In [4]:
test_sql = "SELECT TOP 1000 * FROM HumanResources.Department"
departments = connection.execute(test_sql)
df_departments = pd.DataFrame(departments.fetchall())
df_departments.columns = departments.keys()
df_departments

,DepartmentID,Name,GroupName,ModifiedDate
0,1,Engineering,Research and Development,2008-04-30
1,2,Tool Design,Research and Development,2008-04-30
2,3,Sales,Sales and Marketing,2008-04-30
3,4,Marketing,Sales and Marketing,2008-04-30
4,5,Purchasing,Inventory Management,2008-04-30
5,6,Research and Development,Research and Development,2008-04-30
6,7,Production,Manufacturing,2008-04-30
7,8,Production Control,Manufacturing,2008-04-30
8,9,Human Resources,Executive General and Administration,2008-04-30
9,10,Finance,Executive General and Administration,2008-04-30


In [5]:
connection.close()   # Always close the connection in Jupyter, as the kernel stays running and the connection will as well.